# pCrunch's Crunch class

The `Crunch` class is a general analysis tool for batches of time-series based data across multiple environmental conditions (i.e., a full wind speed and turbulence seed sweep). The methods are agnostic to the aeroelastic multibody simulation tool (OpenFAST or HAWC2 or Bladed or QBlade or in-house equivalents). The `AeroelasticOutput` class provides the data containers for each individual simulation.  The `AeroelasticOutput` class provides many analysis capabilities and the `Crunch` class extends them into their batch versions.

The `Crunch` class supports keeping all time series data in memory and a lean "streaming" version where outputs are processed and then deleted, retaining only the critical statistics and analysis outputs.

This file lays out some workflows and showcases capabilities of the `Crunch` class.  It is probably best to walk through the examples of the `AeroelasticOutput` class first.

## Creating a new class instance

The `Crunch` class can be initialized from a list of AeroelasticOutput instances or none, in order to setup a "streaming" analysis.  Pleaes see the AeroelasticOutput example for the various means to initialize one of its instances.  pCrunch provides a reader for OpenFAST output files (both binary and ascii) and common Python data structures are also supported.  To extend pCrunch for use with other aeroelastic multibody codes, users could simply use the `openfast_readers.py` file as a template.  Here are some examples:

In [45]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pCrunch import Crunch, read, FatigueParams

thisdir = os.path.realpath('')
datadir = os.path.join(thisdir, '..', 'pCrunch', 'test', 'data')

# OpenFAST output files
filelist = glob.glob( os.path.join(datadir, '*.out') )
filelist.sort()
print(f"Found {len(filelist)} files.")

# Read all outputs into a list
outputs = [read(m) for m in filelist[1:]]

# Vector magnitudes
mc = {
    "RootMc1": ["RootMxc1", "RootMyc1", "RootMzc1"],
    "RootMc2": ["RootMxc2", "RootMyc2", "RootMzc2"],
    "RootMc3": ["RootMxc3", "RootMyc3", "RootMzc3"],
}

# Channel-specific fatigue properties
fc = {
    "RootMc1": FatigueParams(slope=10.0, ultimate_stress=6e6, load2stress=5e3, S_intercept=5e7),
    "RootMc2": FatigueParams(slope=10.0, ultimate_stress=6e6, load2stress=5e3, S_intercept=5e7),
    "RootMc3": FatigueParams(slope=10.0, ultimate_stress=6e6, load2stress=5e3, S_intercept=5e7),
}

# Channels to focus on for extreme event tabulation
ec = ["RotSpeed", "RotThrust", "RotTorq"]

# Standard use case with all outputs read prior to use of Crunch.
mycruncher = Crunch(outputs)

# Can also add some batch data operations in the constructor (many more available in Batch Processing below)
mycruncher_mc = Crunch(outputs, magnitude_channels=mc, trim_data=[20, 80], fatigue_channels=fc, extreme_channels=ec)

# When planning on adding outputs later, you still need create a Crunch object that is initially empty of data
# The `lean` flag says that the outputs should be processed, but not stored in memory
mycruncher_lean = Crunch(outputs=[], lean=True)

# Can still add the batch operations to be done later when outputs are added
mycruncher_lean_mc = Crunch(outputs=[], lean=True, magnitude_channels=mc, trim_data=[20,80], fatigue_channels=fc, extreme_channels=ec)

Found 4 files.


## Crunching the data

### With full memory storage
The Crunch class can batch process the outputs using one or more processors up to the number available workstation cores.  This computes the essential statistics for each output.

In [46]:
# Process all outputs in parallel
mycruncher.process_outputs(cores=1)

# Process all outputs and override any prior input setting (especially in fatigue calculation)
mycruncher_mc.process_outputs(return_damage=True)

The key outputs that are stacked together for each output are:

- Summary statistics
- Load ranking
- Extreme event table
- Damage equivalent loads (DELs)
- Palmgren-Miner damage

In [47]:
# The summary stats per each file are here:
mycruncher.summary_stats

,Time,WindVxi,WindVyi,WindVzi,WaveElev,Wave1Vxi,Wave1Vyi,Wave1Vzi,Wave1Axi,Wave1Ayi,...,Fair8Ang,Anch8Ten,Anch8Ang,TipSpdRat,RotCp,RotCt,RotCq,RootMc1,RootMc2,RootMc3
,min,min,min,min,min,min,min,min,min,min,...,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated
DLC2.3_1.out,30.0,8.200,0.0,0.0,-0.812,-0.5420,0.0,-0.7480,-0.7230,0.0,...,4087.5075,0.0,0.0,270.834284,11.814836,28.620454,1.591687,278935.802460,285611.016315,296104.068664
DLC2.3_2.out,30.0,8.197,0.0,0.0,-1.055,-0.5438,0.0,-0.5059,-0.3195,0.0,...,4097.1290,0.0,0.0,273.021550,12.052705,29.113332,1.613299,283522.692262,290846.081369,300151.026876
DLC2.3_3.out,30.0,8.197,0.0,0.0,-1.135,-0.5403,0.0,-0.5156,-0.2482,0.0,...,4096.3295,0.0,0.0,272.465802,11.956125,28.968783,1.604505,282622.891423,289067.747189,299496.852080


In [48]:
# These are indexable by channel, stat:
mycruncher.summary_stats["RootMc1"]

,min,max,std,mean,median,abs,integrated
DLC2.3_1.out,459.805830,9134.167593,2585.431576,5578.251922,6682.387691,9134.167593,278935.802460
DLC2.3_2.out,277.648587,9079.452302,2580.572916,5669.818103,6823.739586,9079.452302,283522.692262
DLC2.3_3.out,347.604352,8986.223847,2576.896763,5651.840860,6911.860762,8986.223847,282622.891423


In [49]:
mycruncher.summary_stats["RootMc1"]['min']

DLC2.3_1.out    459.805830
DLC2.3_2.out    277.648587
DLC2.3_3.out    347.604352
Name: min, dtype: float64

In [50]:
# Or by file
mycruncher.summary_stats.loc["DLC2.3_1.out"]

Time      min               30.000000
WindVxi   min                8.200000
WindVyi   min                0.000000
WindVzi   min                0.000000
WaveElev  min               -0.812000
                            ...      
RotCt     integrated        28.620454
RotCq     integrated         1.591687
RootMc1   integrated    278935.802460
RootMc2   integrated    285611.016315
RootMc3   integrated    296104.068664
Name: DLC2.3_1.out, Length: 952, dtype: float64

In [51]:
# Load rankings are manipulations of the summary statistics table
# All channels and statistics are available
mycruncher.get_load_rankings(['RootMc1'],['max'])

,file,channel,stat,val
0,DLC2.3_1.out,RootMc1,max,9134.167593


In [52]:
# Damage equivalent loads are found here:
mycruncher_mc.dels

,RootMc1,RootMc2,RootMc3
DLC2.3_1.out,5837.238491,5695.643809,4753.915681
DLC2.3_2.out,5923.101625,5744.707943,2095.514946
DLC2.3_3.out,5813.402906,5757.892432,2171.656013


In [53]:
# Palmgren-Miner damage can be viewed with (although it is not computed without a `return_damage=True`
mycruncher_mc.damage

""
DLC2.3_1.out
DLC2.3_2.out
DLC2.3_3.out


In [10]:
# Extreme events table. For each channel, there is a list of the extreme condition for each output case
mycruncher.extremes

{'RotThrust': [{'Time': 51.6,
   'RotThrust': 759.0,
   'RotSpeed': 10.1,
   'RotTorq': 2410.0},
  {'Time': 60.35, 'RotThrust': 786.9, 'RotSpeed': 10.41, 'RotTorq': -1041.0},
  {'Time': 60.35, 'RotThrust': 746.2, 'RotSpeed': 10.48, 'RotTorq': -1046.0}],
 'RotSpeed': [{'Time': 61.8,
   'RotThrust': 369.0,
   'RotSpeed': 11.1,
   'RotTorq': 844.0},
  {'Time': 61.9, 'RotThrust': 367.0, 'RotSpeed': 11.28, 'RotTorq': 159.4},
  {'Time': 61.9, 'RotThrust': 317.0, 'RotSpeed': 11.33, 'RotTorq': 140.8}],
 'RotTorq': [{'Time': 54.45,
   'RotThrust': 546.0,
   'RotSpeed': 10.6,
   'RotTorq': 2650.0},
  {'Time': 54.4, 'RotThrust': 554.0, 'RotSpeed': 10.74, 'RotTorq': 2701.0},
  {'Time': 54.4, 'RotThrust': 575.1, 'RotSpeed': 10.61, 'RotTorq': 2638.0}]}

### Crunching in "lean / streaming" mode

If operating in "lean / streaming" mode, the outputs can either be processed one at a time, or even more lean, the summary statistics themselves can be passed to the `cruncher` object to append to the running list.

In [11]:
# Adding AeroelasticOutput objects in lean / streaming mode
for iout in outputs:
    mycruncher_lean.add_output( iout ) # Each output is processed without retaining the full time series

# Adding statistics incrementally
results_pool = []
for iout in outputs:
    iresults = mycruncher_lean_mc.process_single( iout ) # This could be the result of parallelized function
    results_pool.append( iresults )

# After parallel processing is complete, assemble all the statistic for batch analysis
for iresults in results_pool:
    fname, stats, extremes, dels, damage =  iresults
    mycruncher_lean_mc.add_output_stats(fname, stats, extremes, dels, damage)

In [12]:
# Results are the same as the full-memory approach above
mycruncher_lean.summary_stats["RootMc1"]['min']

DLC2.3_1.out    459.805830
DLC2.3_2.out    277.648587
DLC2.3_3.out    347.604352
Name: min, dtype: float64

In [13]:
mycruncher_lean_mc.dels

,RootMc1,RootMc2,RootMc3
DLC2.3_1.out,5837.238491,5695.643809,4753.915681
DLC2.3_2.out,5923.101625,5744.707943,2095.514946
DLC2.3_3.out,5813.402906,5757.892432,2171.656013


## Integrating outputs with a probability weighting (AEP, Damage, etc)

When running design load cases, not all windspeeds, or other environmental condition, occur with equal likelihood.  pCrunch provides a way to assign a probability to each output.  This probability can then weight a summation to compute annual energy production (AEP), or sum all Palmgren-Miner damages together.  Using a subset of the outputs is also a provided capability.

pCrunch provides a couple different ways to set the probabilities, either:
- Inflow wind speed using a Weibull or Rayleigh distribution for the site
- IEC turbine class with different average wind speeds that define a Weibull distribution
- Users can set the probability values directly.

In [14]:
# Set probability based on wind speed channel name, Weibull distribution average of 7.5 m/s (shape factor input optional)
mycruncher.set_probability_wind_distribution('WindVxi', 7.5, kind='weibull', weibull_k=2.0)

# Or Rayleigh distribution using the same distribution average of 7.5 m/s
mycruncher.set_probability_wind_distribution('WindVxi', 7.5, kind='rayleigh')

# If you only want to use some of the outputs, but not all of them
mycruncher.set_probability_wind_distribution('WindVxi', 7.5, kind='weibull', idx=[0,2])

# If you would rather specify the inflow wind speed directly to use in the probability distribution
mycruncher.set_probability_wind_distribution([8,10,12], 7.5, kind='weibull')

# Can also set the probability based on IEC turbine class, again using a channel name of user input of wind speeds
mycruncher.set_probability_turbine_class('WindVxi', 2)
mycruncher.set_probability_turbine_class([8,10,12], 2)

# A savvy user can set the probability values directly (they will be rescaled to sum to one no matter what)
mycruncher.prob = np.array([0.1, 0.5, 0.4])

Once the probabilities are set, the user can use them to calculate AEP or total fatigue accumulation across the scenarios represented by each output.  For the AEP calculation, the user must specify the channel name.  Additional loss factors or restriction to certain indices are optional inputs.

In [15]:
# Probability weighted and unweighted AEP values are returned
mycruncher.compute_aep('GenPwr')

(46447668302.399994, 46270686167.999985)

In [16]:
# Or with loss factors and restricted by select outputs
mycruncher.compute_aep('GenPwr', loss_factor=0.15, idx=[0,2])

(39291750341.28001, 39160481977.799995)

In [17]:
# Damage calculation does not require a channel name, as it uses the previously computed case-specific and channel-specific values.
dels_tot, dams_tot = mycruncher_mc.compute_total_fatigue(lifetime=30.0)

In [18]:
dels_tot

,RootMc1,RootMc2,RootMc3
Weighted,5857.914341,5732.748061,3007.02888
Unweighted,5857.914341,5732.748061,3007.02888


In [19]:
dams_tot

""
Weighted
Unweighted
Lifetime Weighted
Lifetime Unweighted


In [20]:
# Select indices are also available to restrict the summation
dels_tot, dams_tot = mycruncher_mc.compute_total_fatigue(idx=[0,2])

## Other Batch Procressing

The Crunch class provides batch extensions of nearly all of the operations offered in the AeroelasticOutputs class.  This includes the add channel or drop channel utilities and all statistical functions.  For the statistics, unlike the AeroelasticOutput class, these batch versions are functions, not data properties.  The result is returned as a list, with each index corresponding to the output list.  Many of these statistics also vary by channel, so there are likely to be nested lists.  Also, some are unavailable in "lean / streaming" mode.

In [21]:
# Adding channel
mycruncher.calculate_channel('LSSGagMya + LSSGagMza', 'Test')

# Adding Load Roses
lr = {'TwrBs': ['TwrBsFxt', 'TwrBsFyt']}
mycruncher.add_load_rose(lr, nsec=6)

# Dropping channels by string wildcard
mycruncher.drop_channel('Fair*')
mycruncher.drop_channel('Anch*')
mycruncher.drop_channel('Spn*')
mycruncher.drop_channel('Root*')
mycruncher.drop_channel('Wave*')
mycruncher.drop_channel('Ptfm*')
mycruncher.drop_channel('Tw*')
mycruncher.drop_channel('Yaw*')

Added channel, TwrBs15
Added channel, TwrBs45
Added channel, TwrBs75
Added channel, TwrBs105
Added channel, TwrBs135
Added channel, TwrBs165
Added channel, TwrBs15
Added channel, TwrBs45
Added channel, TwrBs75
Added channel, TwrBs105
Added channel, TwrBs135
Added channel, TwrBs165
Added channel, TwrBs15
Added channel, TwrBs45
Added channel, TwrBs75
Added channel, TwrBs105
Added channel, TwrBs135
Added channel, TwrBs165


In [22]:
# Indices to the minimum value for each channel
mycruncher.idxmins()

array([[  0, 648,   0,   0, 601, 601,   0,   0,   0,   0, 458, 998, 998,
        690, 462, 705, 484, 686, 447, 673, 988, 853, 752, 409, 753, 188,
          0, 722, 486,  99, 606, 707, 689, 998, 653, 755, 606, 692],
       [  0, 650,   0,   0, 601, 601,   0,   0,   0,   0, 573, 998, 998,
        690, 424, 707, 612, 687, 449, 674, 989, 856, 817, 920, 985, 186,
          0, 722, 488, 460, 606, 708, 689, 998, 653, 755, 606, 693],
       [  0, 650,   0,   0, 601, 601,   0,   0,   0,   0, 460, 998, 998,
        690, 631, 705, 612, 686, 449, 674, 988, 854, 817, 921, 753, 187,
          0, 722, 488, 460, 606, 706, 688, 998, 653, 755, 606, 692]])

In [23]:
# Indices to the maximum value for each channel
mycruncher.idxmaxs()

array([[1000,  702,    0,    0,  513,  515,    0,  830,  830,  830,  687,
         636,  638,  564,  877,  457,  665,  434,  763,  484,  712,  755,
         603,  498,  855,  751,    0,  432,  309,  514,  489,  767,  715,
         648,  492,  431,  489,  755],
       [1000,  705,    0,    0,  507,  513,    0,  830,  830,  830,  460,
         638,  638,  563,  883,  459,  666,  435,  765,  485,  711,  983,
         603,  752,  857,  818,    0,  607,  314,  516,  488,  761,  715,
         649,  491,  607,  489,  754],
       [1000,  705,    0,    0,  507,  511,    0,  830,  830,  830,  105,
         638,  638,  800,  886,  462,  666,  438,  764,  488,  711,  982,
         603,  752,  855,  817,    0,  607,   75,  516,  488,  790,  715,
         649,  491,  607,  489,  753]])

In [24]:
# Minimum value of each channel
mycruncher.minima()

array([[ 3.00000000e+01,  8.20000000e+00,  0.00000000e+00,
         0.00000000e+00, -8.12000000e-01, -5.42000000e-01,
         0.00000000e+00, -7.48000000e-01, -7.23000000e-01,
         0.00000000e+00, -8.32000000e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  7.56000000e-02,
        -3.20000000e-02, -3.14000000e+00, -2.13000000e-02,
        -3.07000000e-01, -2.09000000e+00, -9.38000000e-01,
         0.00000000e+00, -3.08000000e+00, -8.94000000e-01,
         0.00000000e+00, -3.44000000e+00, -1.22000000e+00,
         0.00000000e+00,  9.61000000e+00,  9.39000000e+00,
         9.52000000e+00, -2.01000000e+00, -1.32000000e-01,
        -4.79000000e-01, -8.20000000e-01, -5.22000000e-02,
         0.00000000e+00,  1.92000000e+01, -1.29000000e-01,
        -1.01000000e+00, -1.07000000e-01, -5.44000000e+00,
        -1.29000000e+00, -3.51000000e-01, -1.92000000e-02,
        -3.52000000e-01, -1.27000000e+02, -2.12000000e+0

In [25]:
# Maximum value of each channel
mycruncher.maxima()

array([[ 8.00000000e+01,  1.27000000e+01,  0.00000000e+00,
         0.00000000e+00,  1.11000000e+00,  8.75000000e-01,
         0.00000000e+00,  6.70000000e-01,  6.80000000e-01,
         0.00000000e+00,  5.37000000e-01,  2.79000000e+03,
         2.73000000e+01,  0.00000000e+00,  9.00000000e+01,
         9.00000000e+01,  9.00000000e+01,  3.60000000e+02,
         1.11000000e+01,  1.09000000e+03,  6.38000000e-03,
         1.29000000e+00,  4.91000000e+00,  1.23000000e+00,
         0.00000000e+00,  4.58000000e+00,  1.57000000e+00,
         0.00000000e+00,  4.72000000e+00,  2.10000000e+00,
         0.00000000e+00,  6.55000000e+01,  6.54000000e+01,
         6.54000000e+01,  2.19000000e+00,  1.76000000e-01,
         5.19000000e-01,  6.94000000e-01,  2.72000000e-02,
         0.00000000e+00,  3.72000000e+01,  4.21000000e-02,
         6.39000000e-01,  1.93000000e-01,  5.02000000e+00,
         3.11000000e-01,  2.67000000e-01,  1.03000000e-02,
         3.83000000e-01,  2.50000000e+02,  1.70000000e+0

In [26]:
# Maximum value of absolute values of each channel
mycruncher.absmaxima()

array([[8.000e+01, 1.270e+01, 0.000e+00, 0.000e+00, 2.790e+03, 2.730e+01,
        0.000e+00, 9.000e+01, 9.000e+01, 9.000e+01, 3.600e+02, 1.110e+01,
        1.090e+03, 2.130e-02, 1.290e+00, 4.910e+00, 1.230e+00, 4.580e+00,
        1.570e+00, 4.720e+00, 2.100e+00, 2.190e+00, 1.760e-01, 5.190e-01,
        8.200e-01, 5.220e-02, 0.000e+00, 7.590e+02, 1.140e+03, 1.130e+03,
        2.650e+03, 3.840e+03, 3.590e+03, 8.830e+00, 4.660e-01, 1.130e+00,
        6.240e-02, 4.930e+03],
       [8.000e+01, 1.272e+01, 0.000e+00, 0.000e+00, 2.866e+03, 2.780e+01,
        0.000e+00, 9.000e+01, 9.000e+01, 9.000e+01, 3.595e+02, 1.128e+01,
        1.103e+03, 2.337e-02, 1.418e+00, 4.852e+00, 1.152e+00, 4.701e+00,
        1.658e+00, 4.526e+00, 2.183e+00, 2.472e+00, 1.646e-01, 5.903e-01,
        9.164e-01, 5.507e-02, 0.000e+00, 7.869e+02, 1.128e+03, 1.117e+03,
        2.701e+03, 3.969e+03, 3.930e+03, 8.988e+00, 4.801e-01, 1.171e+00,
        6.365e-02, 5.283e+03],
       [8.000e+01, 1.272e+01, 0.000e+00, 0.000e+00

In [27]:
# The range of data values (max - min)
mycruncher.ranges()

array([[5.00000000e+01, 4.50000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.92200000e+00, 1.41700000e+00, 0.00000000e+00, 1.41800000e+00,
        1.40300000e+00, 0.00000000e+00, 1.36900000e+00, 2.79000000e+03,
        2.73000000e+01, 0.00000000e+00, 9.00000000e+01, 9.00000000e+01,
        9.00000000e+01, 3.59924400e+02, 1.11320000e+01, 1.09314000e+03,
        2.76800000e-02, 1.59700000e+00, 7.00000000e+00, 2.16800000e+00,
        0.00000000e+00, 7.66000000e+00, 2.46400000e+00, 0.00000000e+00,
        8.16000000e+00, 3.32000000e+00, 0.00000000e+00, 5.58900000e+01,
        5.60100000e+01, 5.58800000e+01, 4.20000000e+00, 3.08000000e-01,
        9.98000000e-01, 1.51400000e+00, 7.94000000e-02, 0.00000000e+00,
        1.80000000e+01, 1.71100000e-01, 1.64900000e+00, 3.00000000e-01,
        1.04600000e+01, 1.60100000e+00, 6.18000000e-01, 2.95000000e-02,
        7.35000000e-01, 3.77000000e+02, 3.82000000e+02, 5.62000000e+02,
        8.47000000e+03, 1.40400000e+04, 1.38200000e+02, 3.780000

In [28]:
# Channel indices which vary in time
mycruncher.variable()

array([[ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37],
       [ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37],
       [ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37]])

In [29]:
# Channel indices which are constant in time
mycruncher.constant()

array([[ 2,  3,  6, 26],
       [ 2,  3,  6, 26],
       [ 2,  3,  6, 26]])

In [30]:
# Sum of channel values over time
mycruncher.sums()

array([[ 5.50550000e+04,  9.46900000e+03,  0.00000000e+00,
         0.00000000e+00,  1.45390000e+06,  1.49149000e+04,
         0.00000000e+00,  2.55375000e+04,  2.55375000e+04,
         2.55375000e+04,  1.79756283e+05,  7.75227039e+03,
         7.51761046e+05, -3.42743786e-01,  2.02690152e+02,
         2.30346591e+03, -1.09907136e+02,  2.28235849e+03,
         3.55282620e+01,  2.24589427e+03, -3.25890567e+02,
        -3.26346500e+01, -2.43732450e+00, -5.43152870e+01,
         1.85380310e+02, -1.67354519e+01,  0.00000000e+00,
         3.79851510e+05,  1.79681830e+04,  2.15917104e+05,
         1.34039224e+06,  5.52030480e+05, -5.05639300e+04,
         5.42017454e+03,  2.36486217e+02,  5.72687080e+02,
         3.18608147e+01,  5.01466550e+05],
       [ 5.50550000e+04,  9.46870100e+03,  0.00000000e+00,
         0.00000000e+00,  1.48104500e+06,  1.50961200e+04,
         0.00000000e+00,  2.55375000e+04,  2.55375000e+04,
         2.55375000e+04,  1.80977271e+05,  7.81203723e+03,
         7.57

In [31]:
# Sum of channel values over time to the second power
mycruncher.sums_squared()

array([[3.23698375e+06, 9.00038632e+04, 0.00000000e+00, 0.00000000e+00,
        3.52951220e+09, 3.70704810e+05, 0.00000000e+00, 1.99462425e+06,
        1.99462425e+06, 1.99462425e+06, 4.06394804e+07, 7.52037090e+04,
        7.07200564e+08, 1.54658179e-02, 3.39258175e+02, 9.65822856e+03,
        3.18990780e+02, 9.66947390e+03, 4.16324858e+02, 9.84785230e+03,
        5.88479093e+02, 9.15025305e+02, 3.15898576e+00, 6.07529038e+01,
        1.90875171e+02, 6.53175358e-01, 0.00000000e+00, 2.53852167e+08,
        5.24725336e+08, 6.12385416e+08, 3.59798262e+09, 2.73503631e+09,
        2.19458126e+09, 3.69968821e+04, 1.00909306e+02, 5.60614252e+02,
        1.99075376e+00, 4.13739973e+09],
       [3.23698375e+06, 8.99982441e+04, 0.00000000e+00, 0.00000000e+00,
        3.66867825e+09, 3.80048993e+05, 0.00000000e+00, 1.99462425e+06,
        1.99462425e+06, 1.99462425e+06, 4.11235063e+07, 7.63658754e+04,
        7.18650285e+08, 1.84534584e-02, 4.11020402e+02, 9.93242786e+03,
        3.29216945e+02,

In [32]:
# Sum of channel values over time to the third power
mycruncher.sums_cubed()

array([[ 2.01019569e+08,  8.60415126e+05,  0.00000000e+00,
         0.00000000e+00,  8.60044461e+12,  9.22847278e+06,
         0.00000000e+00,  1.65847398e+08,  1.65847398e+08,
         1.65847398e+08,  1.02599568e+10,  7.54261954e+05,
         6.87823481e+11, -1.55308989e-04,  3.65346996e+02,
         3.65916264e+04, -2.55926991e+01,  3.57713395e+04,
         1.76569145e+02,  3.58096549e+04,  2.81610848e+01,
         9.81213119e+01, -2.58180656e-02, -4.73394835e+00,
         2.46636284e+01, -2.05330497e-02,  0.00000000e+00,
         1.42703393e+11, -3.28745590e+10,  1.74268808e+11,
         8.42326328e+12,  1.53078768e+12,  5.92738091e+11,
         2.62563216e+05,  4.04494814e+01,  4.72940081e+02,
         1.10493376e-01,  4.07709725e+12],
       [ 2.01019569e+08,  8.60336688e+05,  0.00000000e+00,
         0.00000000e+00,  9.13692517e+12,  9.59022632e+06,
         0.00000000e+00,  1.65847398e+08,  1.65847398e+08,
         1.65847398e+08,  1.04146405e+10,  7.71699300e+05,
         7.04

In [33]:
# Sum of channel values over time to the fourth power
mycruncher.sums_fourth()

array([[1.30308931e+10, 8.28208508e+06, 0.00000000e+00, 0.00000000e+00,
        2.10406009e+16, 2.30121389e+08, 0.00000000e+00, 1.41881502e+10,
        1.41881502e+10, 1.41881502e+10, 2.78473049e+12, 7.64467456e+06,
        6.76030490e+14, 2.75371793e-06, 4.38130836e+02, 1.47734268e+05,
        2.02766160e+02, 1.44888241e+05, 4.39773166e+02, 1.48091086e+05,
        9.32527616e+02, 2.23246918e+03, 2.51710354e-02, 7.55625804e+00,
        6.42001326e+01, 8.23896878e-04, 0.00000000e+00, 9.06369347e+13,
        4.28375901e+14, 5.18043037e+14, 2.05603252e+16, 1.39022813e+16,
        1.08541985e+16, 1.89109701e+06, 1.66195253e+01, 4.45136610e+02,
        6.33486427e-03, 4.18515574e+16],
       [1.30308931e+10, 8.28112759e+06, 0.00000000e+00, 0.00000000e+00,
        2.28840158e+16, 2.42584716e+08, 0.00000000e+00, 1.41881502e+10,
        1.41881502e+10, 1.41881502e+10, 2.83011774e+12, 7.88149761e+06,
        6.98059498e+14, 4.09761504e-06, 6.40751982e+02, 1.55841113e+05,
        2.02329934e+02,

In [34]:
# Second moment of the timeseries for each channel
mycruncher.second_moments()

array([[2.08750000e+02, 4.31043545e-01, 0.00000000e+00, 0.00000000e+00,
        1.41638232e+06, 1.48324476e+02, 0.00000000e+00, 1.34177009e+03,
        1.34177009e+03, 1.34177009e+03, 8.35108819e+03, 1.51508995e+01,
        1.42477996e+05, 1.53331288e-05, 2.97918002e-01, 4.35322077e+00,
        3.06616653e-01, 4.46105650e+00, 4.14649213e-01, 4.80404619e+00,
        4.81898632e-01, 9.13048301e-01, 3.14990124e-03, 5.77479526e-02,
        1.56387256e-01, 3.73006798e-04, 0.00000000e+00, 1.09599541e+05,
        5.23878924e+05, 5.65246548e+05, 1.80132480e+06, 2.42817492e+06,
        2.18983727e+06, 7.64029872e+00, 4.49944503e-02, 2.32738665e-01,
        9.75680662e-04, 3.88229995e+06],
       [2.08750000e+02, 4.31081130e-01, 0.00000000e+00, 0.00000000e+00,
        1.47589936e+06, 1.52231588e+02, 0.00000000e+00, 1.34177009e+03,
        1.34177009e+03, 1.34177009e+03, 8.39505866e+03, 1.53835331e+01,
        1.44801309e+05, 1.83076546e-05, 3.61217221e-01, 4.44220182e+00,
        3.19316816e-01,

In [35]:
# Third moment of the timeseries for each channel
mycruncher.third_moments()

array([[ 2.32598046e-13,  8.55981766e-01,  0.00000000e+00,
         0.00000000e+00, -6.43899356e+08, -7.18799532e+02,
         0.00000000e+00,  4.63832777e+04,  4.63832777e+04,
         4.63832777e+04, -4.02281607e+04, -6.29998557e+01,
        -5.74526007e+07, -1.39363439e-07,  1.75705605e-01,
        -5.68285719e+00,  7.67536023e-02, -6.63264695e+00,
         1.32196894e-01, -7.85638822e+00,  5.33308410e-01,
         1.87359675e-01, -2.76885221e-06,  4.83092886e-03,
        -6.85991578e-02,  2.86922017e-06,  0.00000000e+00,
        -3.68526562e+07, -6.10587470e+07, -2.01714665e+08,
        -1.22236801e+09, -2.65572482e+09,  9.24123321e+08,
        -2.05689497e+01, -4.66680665e-03, -1.14254321e-01,
        -1.50272309e-05, -1.88739735e+09],
       [ 2.32598046e-13,  8.57125048e-01,  0.00000000e+00,
         0.00000000e+00, -6.62208883e+08, -7.36783196e+02,
         0.00000000e+00,  4.63832777e+04,  4.63832777e+04,
         4.63832777e+04, -5.89151627e+04, -6.45666862e+01,
        -5.89

In [36]:
# Fourth moment of the timeseries for each channel
mycruncher.fourth_moments()

array([[7.84377081e+04, 2.80590766e+00, 0.00000000e+00, 0.00000000e+00,
        2.38203892e+12, 2.58664085e+04, 0.00000000e+00, 3.77719781e+06,
        3.77719781e+06, 3.77719781e+06, 1.55099537e+08, 5.39036464e+02,
        4.76716441e+10, 2.54929447e-09, 2.20409100e-01, 3.35434083e+01,
        2.13949206e-01, 3.90565302e+01, 4.17430017e-01, 4.80097782e+01,
        1.30840176e+00, 2.24884821e+00, 2.50068383e-05, 7.56841574e-03,
        8.15947135e-02, 3.11255202e-07, 0.00000000e+00, 3.10555146e+10,
        4.31319128e+11, 5.31605607e+11, 4.49263428e+12, 1.52233325e+13,
        1.09965456e+13, 1.31007075e+02, 2.82991207e-03, 1.41949006e-01,
        1.28472121e-06, 3.96828841e+13],
       [7.84377081e+04, 2.81289319e+00, 0.00000000e+00, 0.00000000e+00,
        2.60258994e+12, 2.73210280e+04, 0.00000000e+00, 3.77719781e+06,
        3.77719781e+06, 3.77719781e+06, 1.54959125e+08, 5.57948905e+02,
        4.94241367e+10, 3.80067779e-09, 3.18002721e-01, 3.44805158e+01,
        2.11290221e-01,

In [37]:
# Mean of channel values over time
mycruncher.means()

array([[ 5.50000000e+01,  9.45954046e+00,  0.00000000e+00,
         0.00000000e+00,  2.35944635e-02,  1.34231598e-02,
         0.00000000e+00, -5.43743856e-03, -4.11430669e-03,
         0.00000000e+00, -7.61653746e-03,  1.45244755e+03,
         1.49000000e+01,  0.00000000e+00,  2.55119880e+01,
         2.55119880e+01,  2.55119880e+01,  1.79576706e+02,
         7.74452587e+00,  7.51010036e+02, -3.42401385e-04,
         2.02487664e-01,  2.30116475e+00, -1.09797339e-01,
         0.00000000e+00,  2.28007842e+00,  3.54927692e-02,
         0.00000000e+00,  2.24365062e+00, -3.25565002e-01,
         0.00000000e+00,  4.83947453e+01,  5.49971528e+01,
         5.61357742e+01, -3.26020480e-02, -2.43488961e-03,
        -5.42610260e-02,  1.85195115e-01, -1.67187331e-02,
         0.00000000e+00,  3.08333666e+01, -3.35188573e-02,
        -1.17889801e-01,  4.97514717e-02,  6.40367922e-01,
        -2.02175950e-01, -5.23522243e-02, -4.82075524e-05,
        -5.69667832e-03,  1.23310600e+02, -1.19347253e+0

In [38]:
# Median of channel values over time
mycruncher.medians()

array([[ 5.50000000e+01,  9.40000000e+00,  0.00000000e+00,
         0.00000000e+00,  7.53000000e-03, -1.38000000e-02,
         0.00000000e+00, -3.18000000e-02, -2.99000000e-02,
         0.00000000e+00,  3.72000000e-02,  2.31000000e+03,
         2.41000000e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.90000000e+02,
         1.00000000e+01,  9.72000000e+02,  8.56000000e-05,
        -8.89000000e-03,  3.54000000e+00, -1.54000000e-01,
         0.00000000e+00,  3.55000000e+00, -5.55000000e-03,
         0.00000000e+00,  3.53000000e+00, -4.82000000e-01,
         0.00000000e+00,  5.63000000e+01,  6.37000000e+01,
         6.34000000e+01, -1.01000000e-01,  1.28000000e-03,
        -6.91000000e-02,  3.19000000e-01, -2.01000000e-02,
         0.00000000e+00,  3.22000000e+01, -2.62000000e-02,
        -9.35000000e-02,  5.32000000e-02,  1.18000000e+00,
         8.47000000e-03, -4.07000000e-02, -9.95000000e-05,
        -4.34000000e-03,  1.83000000e+02,  4.87000000e+0

In [39]:
# Standard deviation of channel values over time
mycruncher.stddevs()

array([[1.44481833e+01, 6.56539066e-01, 0.00000000e+00, 0.00000000e+00,
        4.06182652e-01, 2.99791180e-01, 0.00000000e+00, 2.99704444e-01,
        2.88576051e-01, 0.00000000e+00, 2.90032670e-01, 1.19011862e+03,
        1.21788536e+01, 0.00000000e+00, 3.66301800e+01, 3.66301800e+01,
        3.66301800e+01, 9.13842885e+01, 3.89241564e+00, 3.77462575e+02,
        3.91575392e-03, 5.45818653e-01, 2.08643734e+00, 5.53729765e-01,
        0.00000000e+00, 2.11212133e+00, 6.43932615e-01, 0.00000000e+00,
        2.19181345e+00, 6.94189190e-01, 0.00000000e+00, 1.76987733e+01,
        1.59996408e+01, 1.49473147e+01, 9.55535609e-01, 5.61239810e-02,
        2.40308037e-01, 3.95458286e-01, 1.93133839e-02, 0.00000000e+00,
        5.74805927e+00, 5.58476561e-02, 3.47536919e-01, 7.30338449e-02,
        2.45823718e+00, 5.46411299e-01, 1.40582067e-01, 4.02500115e-03,
        1.52217339e-01, 9.97630831e+01, 1.19716420e+02, 1.67375256e+02,
        2.44801391e+03, 3.75096697e+03, 2.97634139e+01, 1.049732

In [40]:
# Skew of channel values over time
mycruncher.skews()

/Users/gbarter/devel/pCrunch/pCrunch/aeroelastic_output.py:441: RuntimeWarning: invalid value encountered in divide
  return self.third_moments / np.sqrt(self.second_moments) ** 3


array([[ 7.71198782e-17,  3.02470340e+00,             nan,
                    nan, -3.81985823e-01, -3.97913390e-01,
                    nan,  9.43722429e-01,  9.43722429e-01,
         9.43722429e-01, -5.27127376e-02, -1.06827220e+00,
        -1.06828713e+00, -2.32114718e+00,  1.08053912e+00,
        -6.25677738e-01,  4.52069451e-01, -7.03931330e-01,
         4.95108046e-01, -7.46126001e-01,  1.59420766e+00,
         2.14751148e-01, -1.56622578e-02,  3.48117419e-01,
        -1.10921758e+00,  3.98280235e-01,             nan,
        -1.01567717e+00, -1.61028037e-01, -4.74657772e-01,
        -5.05607996e-01, -7.01880406e-01,  2.85175471e-01,
        -9.73972305e-01, -4.88969063e-01, -1.01758296e+00,
        -4.93080029e-01, -2.46734404e-01],
       [ 7.71198782e-17,  3.02834721e+00,             nan,
                    nan, -3.69326003e-01, -3.92267699e-01,
                    nan,  9.43722429e-01,  9.43722429e-01,
         9.43722429e-01, -7.65934229e-02, -1.07009999e+00,
        -1.06

In [41]:
# Kurtosis of channel values over time
mycruncher.kurtosis()

/Users/gbarter/devel/pCrunch/pCrunch/aeroelastic_output.py:445: RuntimeWarning: invalid value encountered in divide
  return self.fourth_moments / self.second_moments ** 2


array([[ 1.7999976 , 15.10188237,         nan,         nan,  1.18737488,
         1.17573782,         nan,  2.09803882,  2.09803882,  2.09803882,
         2.22394663,  2.34823366,  2.34835925, 10.84322418,  2.48333911,
         1.77005109,  2.27572198,  1.96253861,  2.42785075,  2.08024914,
         5.63416724,  2.69756946,  2.52037354,  2.26950841,  3.33625549,
         2.23708829,         nan,  2.58536139,  1.57158134,  1.66384826,
         1.38457666,  2.5819617 ,  2.29315069,  2.24426466,  1.3978322 ,
         2.62056706,  1.34956404,  2.63284357],
       [ 1.7999976 , 15.13683985,         nan,         nan,  1.19479187,
         1.17892854,         nan,  2.09803882,  2.09803882,  2.09803882,
         2.19871878,  2.35766582,  2.35718742, 11.33954508,  2.43721555,
         1.74733905,  2.07221979,  2.03145391,  2.72636582,  2.11509162,
         5.95544468,  3.3735226 ,  2.59843189,  2.42702667,  3.64709245,
         2.64238012,         nan,  2.7570548 ,  1.54705475,  1.67612674,
   

In [42]:
# Integration of channel values over time
mycruncher.integrated()

array([[ 2.75000000e+03,  4.72980000e+02,  0.00000000e+00,
         0.00000000e+00,  1.18250290e+00,  6.65579150e-01,
         0.00000000e+00, -2.70943800e-01, -2.05831050e-01,
         0.00000000e+00, -3.69707700e-01,  7.26372500e+04,
         7.45140000e+02,  0.00000000e+00,  1.27462500e+03,
         1.27462500e+03,  1.27462500e+03,  8.98084913e+03,
         3.87364817e+02,  3.75639053e+04, -1.71749143e-02,
         1.01104651e+01,  1.15075058e+02, -5.49288180e+00,
         0.00000000e+00,  1.14034950e+02,  1.77706310e+00,
         0.00000000e+00,  1.12210488e+02, -1.62660283e+01,
         0.00000000e+00,  2.41950200e+03,  2.75072250e+03,
         2.80643800e+03, -1.66878250e+00, -1.21686225e-01,
        -2.72104185e+00,  9.27646550e+00, -8.36064343e-01,
         0.00000000e+00,  1.54224250e+03, -1.67539381e+00,
        -5.88358455e+00,  2.48946116e+00,  3.21374145e+01,
        -1.00948213e+01, -2.61338382e+00, -2.32953800e-03,
        -2.86384750e-01,  6.16741555e+03, -5.96033000e+0

In [43]:
# Special instance of the integration that specifically uses
# the Power channel string to integrate over time and calculate energy
mycruncher.compute_energy('GenPwr')

DLC2.3_1.out    72637.250
DLC2.3_2.out    73994.400
DLC2.3_3.out    73453.425
Name: integrated, dtype: float64

In [44]:
# Total travel across simulation- useful for pitch drives and yaw drivers
mycruncher.total_travel('BldPitch1')

array([90., 90., 90.])